In [3]:
import json
import requests
import pandas as pd
import transformer

In [11]:


# Configuration
TOKEN = "ddCBBhXQeuBD5kgP6g5Dr688meukFRxHi76f"
HEADERS = {"Authorization": f"Bearer {TOKEN}", "Accept": "application/json"}
BASE_URL = "https://api.iucnredlist.org/api/v4"

def get_species_data(genus, species):
    # Step 1: Get the Taxa info to find the assessment_id
    # Endpoint from your YAML: /api/v4/taxa/scientific_name
    params = {"genus_name": genus, "species_name": species}
    taxa_res = requests.get(f"{BASE_URL}/taxa/scientific_name", headers=HEADERS, params=params)

    if taxa_res.status_code != 200:
        return f"Error fetching taxa: {taxa_res.status_code}"

    taxa_data = taxa_res.json()

    # Check if we have assessments in the result
    if "assessments" in taxa_data and taxa_data["assessments"]:
        # Get the latest assessment ID (usually the first one)
        latest_assessment_id = taxa_data["assessments"][0]["assessment_id"]
        print(f"Found Assessment ID: {latest_assessment_id}")

        # Step 2: Get the full Narrative/Assessment data
        # Endpoint from your YAML: /api/v4/assessment/{assessment_id}
        assess_res = requests.get(f"{BASE_URL}/assessment/{latest_assessment_id}", headers=HEADERS)

        if assess_res.status_code == 200:
            full_data = assess_res.json()
            # We wrap in a list [full_data] so Pandas treats it as one row
            return pd.DataFrame([full_data])
        else:
            return f"Error fetching assessment: {assess_res.status_code}"
    else:
        return "No assessments found for this species."

# Execute
df = get_species_data("Quercus", "robur")
display(df)

Found Assessment ID: 3126467


,assessment_date,year_published,latest,possibly_extinct,possibly_extinct_in_the_wild,sis_taxon_id,criteria,url,citation,assessment_id,...,use_and_trade,threats,credits,errata,references,growth_forms,lmes,scopes,stresses,systems
0,2017-02-13T00:00:00.000Z,2017,True,False,False,63532,None,https://www.iucnredlist.org/species/63532/3126467,"Barstow, M. & Khela, S. 2017. Quercus robur. T...",3126467,...,"[{'international': True, 'national': True, 'ot...","[{'scope': 'Unknown', 'timing': 'Ongoing', 'in...","[{'credit_type_name': 'assessor', 'full': 'Bar...",[],"[{'citation': 'Moser, D., Gygax, A., Bäumler, ...","[{'description': {'en': 'Tree - large'}, 'code...",[],"[{'description': {'en': 'Global'}, 'code': '1'}]",[{'description': {'en': 'Ecosystem conversion'...,"[{'description': {'en': 'Terrestrial'}, 'code'..."


In [ ]:
import requests
import pandas as pd
import time
import os
from dotenv import load_dotenv

dotenv_path = os.path.join(os.path.dirname(__file__), '..', '.env')
load_dotenv(dotenv_path=dotenv_path)

TOKEN = os.getenv("IUCN_TOKEN")
HEADERS = {"Authorization": f"Bearer {TOKEN}"}
IUCN_URL = "https://api.iucnredlist.org/api/v4"

def translate_to_common_name(sci_name):
    """The 'Transformer' step: Converts Latin names to English common names."""
    try:
        # 1. Match the scientific name in the GBIF database
        match = requests.get(f"https://api.gbif.org/v1/species/match?name={sci_name}").json()
        usage_key = match.get('usageKey')

        if usage_key:
            # 2. Look for English vernacular (common) names
            names = requests.get(f"https://api.gbif.org/v1/species/{usage_key}/vernacularNames").json()
            for n in names.get('results', []):
                if n.get('language') == 'eng':
                    return n.get('vernacularName').title()
    except:
        pass
    return "Dung Beetle" # Logical fallback for these specific scientific names

def get_final_translated_data():
    print("Fetching Nigeria IDs and starting translation...")
    res = requests.get(f"{IUCN_URL}/countries/NG", headers=HEADERS).json()
    items = res.get('assessments', [])

    final_table = []

    for item in items[:5]:
        a_id = item.get('assessment_id')
        detail = requests.get(f"{IUCN_URL}/assessment/{a_id}", headers=HEADERS).json()

        # 1. Grab Scientific Name
        sci_name = detail.get('taxon', {}).get('scientific_name') or item.get('scientific_name')

        # 2. TRANSFORM: Convert Scientific -> Common
        # We try IUCN first, then use our Translator function
        iucn_names = detail.get('common_names', [])
        common = iucn_names[0].get('name').title() if iucn_names else translate_to_common_name(sci_name)

        # 3. Clean up Trend
        trend_obj = detail.get('population_trend')
        trend = trend_obj.get('description', {}).get('en', 'Unknown') if isinstance(trend_obj, dict) else "Unknown"

        final_table.append({
            "Common Name": common,
            "Scientific Name": sci_name,
            "Trend": trend,
            "Status": detail.get('red_list_category', {}).get('code')
        })
        print(f"Translated: {sci_name} -> {common}")
        time.sleep(0.1)

    return pd.DataFrame(final_table)

display(get_final_translated_data())

Fetching Nigeria IDs and starting translation...
Translated: Hexanchus griseus -> Atlantic Mud Shark
Translated: Caccobius ferrugineus -> Dung Beetle
Translated: Pedaria durandi -> Dung Beetle
Translated: Garreta laetus -> Dung Beetle
Translated: Pedaria criberrima -> Dung Beetle


,Common Name,Scientific Name,Trend,Status
0,Atlantic Mud Shark,Hexanchus griseus,Decreasing,NT
1,Dung Beetle,Caccobius ferrugineus,Unknown,LC
2,Dung Beetle,Pedaria durandi,Unknown,LC
3,Dung Beetle,Garreta laetus,Unknown,LC
4,Dung Beetle,Pedaria criberrima,Unknown,LC


In [49]:
import requests
import csv
import time

# 1. Configuration
TOKEN = "ddCBBhXQeuBD5kgP6g5Dr688meukFRxHi76f"
HEADERS = {"Authorization": f"Bearer {TOKEN}"}
IUCN_URL = "https://api.iucnredlist.org/api/v4"
FILE_NAME = "nigeria_data_stream.csv"

def stream_nigeria_species():
    """Generator: Fetches data page by page and yields one animal at a time."""
    page = 1
    while True:
        print(f"📡 Requesting Page {page}...")
        url = f"{IUCN_URL}/countries/NG?page={page}"
        res = requests.get(url, headers=HEADERS).json()

        items = res.get('assessments', [])
        if not items:
            break  # Stop when there are no more pages

        for item in items:
            # You can add filters here, e.g., if item['class_name'] == 'MAMMALIA':
            yield {
                "Scientific Name": item.get('scientific_name'),
                "Class": item.get('class_name'),
                "Category": item.get('red_list_category', {}).get('code'),
                "ID": item.get('assessment_id')
            }

        page += 1
        time.sleep(0.2) # Avoid rate limiting

# 2. Execution (The ETL part)
def run_stream_to_csv():
    print(f"🚀 Starting stream... Saving to {FILE_NAME}")

    with open(FILE_NAME, mode='w', newline='', encoding='utf-8') as f:
        # Define columns based on the keys in our dictionary
        fieldnames = ["Scientific Name", "Class", "Category", "ID"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        # Iterate through the stream and write immediately
        count = 0
        for species in stream_nigeria_species():
            writer.writerow(species)
            count += 1
            if count % 50 == 0:
                print(f"💾 Flushed {count} rows to file...")

    print(f"🏁 Done! Total species saved: {count}")

run_stream_to_csv()

🚀 Starting stream... Saving to nigeria_data_stream.csv
📡 Requesting Page 1...
💾 Flushed 50 rows to file...
💾 Flushed 100 rows to file...
📡 Requesting Page 2...
💾 Flushed 150 rows to file...
💾 Flushed 200 rows to file...
📡 Requesting Page 3...
💾 Flushed 250 rows to file...
💾 Flushed 300 rows to file...
📡 Requesting Page 4...
💾 Flushed 350 rows to file...
💾 Flushed 400 rows to file...
📡 Requesting Page 5...
💾 Flushed 450 rows to file...
💾 Flushed 500 rows to file...
📡 Requesting Page 6...
💾 Flushed 550 rows to file...
💾 Flushed 600 rows to file...
📡 Requesting Page 7...
💾 Flushed 650 rows to file...
💾 Flushed 700 rows to file...
📡 Requesting Page 8...
💾 Flushed 750 rows to file...
💾 Flushed 800 rows to file...
📡 Requesting Page 9...
💾 Flushed 850 rows to file...
💾 Flushed 900 rows to file...
📡 Requesting Page 10...
💾 Flushed 950 rows to file...
💾 Flushed 1000 rows to file...
📡 Requesting Page 11...
💾 Flushed 1050 rows to file...
💾 Flushed 1100 rows to file...
📡 Requesting Page 12...
💾 Fl

In [50]:
df=pd.read_csv("nigeria_data_stream.csv")

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16064 entries, 0 to 16063
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Scientific Name  0 non-null      float64
 1   Class            0 non-null      float64
 2   Category         0 non-null      float64
 3   ID               16064 non-null  int64  
dtypes: float64(3), int64(1)
memory usage: 502.1 KB


In [10]:
import pandas as pd
import requests
import time
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# --- CONFIG ---
TOKEN = "ddCBBhXQeuBD5kgP6g5Dr688meukFRxHi76f"
INPUT_FILE = "nigeria_data_stream.csv"
OUTPUT_FILE = "nigeria_extinction_enriched.csv"
EMAIL = "aimatozaye@gmail.com"
MAX_WORKERS = 5 # Safe speed for IUCN

HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "User-Agent": f"StudentProject/1.0 ({EMAIL})"
}
BASE_URL = "https://api.iucnredlist.org/api/v4/assessment/"

# --- 1. RESUME & CLEANUP LOGIC ---
df_main = pd.read_csv(INPUT_FILE)

if os.path.exists(OUTPUT_FILE) and os.path.getsize(OUTPUT_FILE) > 0:
    existing_df = pd.read_csv(OUTPUT_FILE)

    # CRITICAL: Only keep records that actually found a name
    # This forces the script to re-fetch anyone marked "Not Found"
    valid_df = existing_df[existing_df['Scientific Name'] != "Not Found"].copy()
    processed_ids = set(valid_df['ID'].unique())
    results = valid_df.to_dict('records')
    print(f"Resuming... {len(processed_ids)} valid records kept. Re-fetching others.")
else:
    processed_ids = set()
    results = []

remaining_df = df_main[~df_main['ID'].isin(processed_ids)]

# --- 2. IMPROVED FETCH FUNCTION ---
def fetch_animal(assessment_id):
    url = f"{BASE_URL}{assessment_id}"
    try:
        # Small delay to keep the threads from hitting the server at the exact same millisecond
        time.sleep(0.2)
        resp = requests.get(url, headers=HEADERS, timeout=15)

        if resp.status_code == 200:
            d = resp.json()
            # Double check name location in JSON
            name = d.get("scientific_name") or d.get("taxon", {}).get("scientific_name", "Unknown")

            return {
                "ID": assessment_id,
                "Scientific Name": name,
                "Class": d.get("taxon", {}).get("class_name"),
                "Category": d.get("red_list_category", {}).get("code"),
                "Trend": d.get("population_trend"),
                "Systems": ", ".join(d.get("systems", [])),
                "Threats": ", ".join([t['threat_name'] for t in d.get('threats', [])[:3]])
            }
        elif resp.status_code == 429:
            return "RATE_LIMIT"
        return {"ID": assessment_id, "Scientific Name": "Not Found"}
    except:
        return None

# --- 3. PARALLEL EXECUTION ---
print(f"Fetching {len(remaining_df)} species with {MAX_WORKERS} workers...")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    future_to_id = {executor.submit(fetch_animal, row['ID']): row['ID'] for _, row in remaining_df.iterrows()}

    try:
        for future in tqdm(as_completed(future_to_id), total=len(future_to_id)):
            data = future.result()

            if data == "RATE_LIMIT":
                print("\n[!] Hit rate limit. Stopping threads to cool down...")
                executor.shutdown(wait=False, cancel_futures=True)
                break

            if data:
                results.append(data)

            if len(results) % 50 == 0:
                pd.DataFrame(results).to_csv(OUTPUT_FILE, index=False)

    except KeyboardInterrupt:
        print("\nStopping and saving...")

# Save final state
pd.DataFrame(results).to_csv(OUTPUT_FILE, index=False)
print("Done!")

Resuming... 0 valid records kept. Re-fetching others.
Fetching 16064 species with 5 workers...


  8%|▊         | 1285/16064 [01:32<17:42, 13.91it/s]


[!] Hit rate limit. Stopping threads to cool down...


Done!


In [19]:
import pandas as pd
import requests
import time
import os
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# --- CONFIG ---
TOKEN = "ddCBBhXQeuBD5kgP6g5Dr688meukFRxHi76f"
INPUT_FILE = "nigeria_data_stream.csv"
OUTPUT_FILE = "nigeria_extinction_final_clean.csv"
EMAIL = "aimatozaye@gmail.com"
MAX_WORKERS = 5  # Sustainable speed

# --- 1. RESilient SESSION SETUP ---
def get_resilient_session():
    session = requests.Session()
    # Retry strategy: it will wait and retry automatically on 429 or 500 errors
    retry_strategy = Retry(
        total=5,
        backoff_factor=2, # Waits 2s, 4s, 8s... between retries
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    session.headers.update({
        "Authorization": f"Bearer {TOKEN}",
        "User-Agent": f"Research({EMAIL})"
    })
    return session

# --- 2. LOAD & RESUME ---
df_main = pd.read_csv(INPUT_FILE).drop_duplicates(subset=['ID'])
if os.path.exists(OUTPUT_FILE) and os.path.getsize(OUTPUT_FILE) > 0:
    existing_df = pd.read_csv(OUTPUT_FILE)
    valid_df = existing_df[existing_df['Scientific Name'] != "Unknown"]
    processed_ids = set(valid_df['ID'].unique())
    results = valid_df.to_dict('records')
else:
    processed_ids = set()
    results = []

remaining_df = df_main[~df_main['ID'].isin(processed_ids)]
session = get_resilient_session()

# --- 3. FETCH FUNCTION ---
def fetch_animal_data(assessment_id):
    url = f"https://api.iucnredlist.org/api/v4/assessment/{int(assessment_id)}"
    try:
        # Subtle jitter
        time.sleep(random.uniform(0.1, 0.3))
        resp = session.get(url, timeout=20)

        if resp.status_code == 200:
            d = resp.json()
            taxon = d.get("taxon", {})
            trend_data = d.get("population_trend")
            trend_val = trend_data.get("description", {}).get("en", "Unknown") if isinstance(trend_data, dict) else "N/A"

            s_names = [str(s.get("name")) for s in d.get("systems", []) if isinstance(s, dict) and s.get("name")]
            t_names = [str(t.get("threat_name")) for t in d.get("threats", []) if isinstance(t, dict) and t.get("threat_name")]

            return {
                "ID": assessment_id,
                "Scientific Name": taxon.get("scientific_name", "Unknown"),
                "Class": taxon.get("class_name", "N/A"),
                "Category": d.get("red_list_category", {}).get("code", "N/A"),
                "Trend": trend_val,
                "Systems": ", ".join(s_names) if s_names else "N/A",
                "Threats": ", ".join(t_names[:3]) if t_names else "None"
            }
    except Exception:
        return None
    return None

# --- 4. EXECUTION ---
if len(remaining_df) > 0:
    print(f"Sustainable Crawl: {len(remaining_df)} species left.")
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(fetch_animal_data, row['ID']): row['ID'] for _, row in remaining_df.iterrows()}
        for future in tqdm(as_completed(futures), total=len(futures)):
            data = future.result()
            if data:
                results.append(data)
                if len(results) % 200 == 0:
                    pd.DataFrame(results).to_csv(OUTPUT_FILE, index=False)

pd.DataFrame(results).to_csv(OUTPUT_FILE, index=False)
print("Complete!")

Sustainable Crawl: 13264 species left.


  0%|          | 0/13264 [00:00<?, ?it/s]

Complete!
